We'll use pandas and numpy heavily for interacting with the data. Matplotlib is used for plotting although there are several higher-level libraries built on it.

ESTIMATE TOTAL MEMORY USAGE: 1200 MB

In [7]:
%%bash
cd ../data/
tar xzf walmart_data.tar.gz

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# Goals of this notebook

* Introduce you to the dataset we'll be using over the next few days. Understanding what problem you are solving is the most important part of any scientific work and clearly defining what one is trying to do is a strict prerequisite to making progress.

* Introduce the bare minimum pandas you'll need to work with tabular data (stuff you can store in a CSV).

**Notes**: You can install extensions (google for nbextensions) with Jupyter notebooks. I tend to use resuse to display memory usage in the top right corner which really helps.

To run a cell, press: "Shift + Enter"

To add a cell before your current cell, press: "Esc + a"

To add a cell after your current cell, press: "Esc + b"

To delete a cell, press: "Esc + x"

To be able to edit a cell, press: "Enter"

To see more documentation about of a function, type ?function_name

To see source code, type ??function_name

To quickly see possible arguments for a function, type "Shift + Tab" after typing the function name.

Esc and Enter take you into different modes. Press "Esc + h" to see all shortcuts.

# Walmart TripType Classification

We have a dataset consisting of shopping trips made to the American chain Walmart. Each trip consists of the following fields:

* **TripType** - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category. Walmart doesn't tell us what the numbers actually mean.
    
* **VisitNumber** - an id corresponding to a single trip by a single customer. We have no information on who the customer was.

* **Weekday** - the weekday of the trip

* **Upc** - the UPC number of the product purchased

* **ScanCount** - the number of the given item that was purchased. A negative value indicates a product return.

* **DepartmentDescription** - a high-level description of the item's department

* **FinelineNumber** - a more refined category for each of the products, created by Walmart

The final task is to predict TripType for each row but we need to review a few things today before we get to the prediction task tomorrow.

In [ ]:
#kaggle provides a train and test dataset
# train = TripType known
# test = TripType unknown
# final task = train model on train and predict TripType on test
# submit answers to kaggle to receive score telling you how well you did

DATA_PATH = "../data/walmart"
TRAIN_PATH = f"{DATA_PATH}/train.csv"
TEST_PATH = f"{DATA_PATH}/test.csv"

### Reading files

Pandas has functionality to read data from and write data to various formats including:

CSV

JSON

Excel (useful for many enterprises heavily reliant on Excel)

Parquet

SQL (write a query)

Feather format (very fast reads/writes)

Each read_{format} function comes with multiple options. For example, you can type "read_csv(" in a code cell below and press Shift + Tab to see all the available options.

In [ ]:
#read csv
df = pd.read_csv(TRAIN_PATH)

In [ ]:
#look at the first few rows
df.head()

In [ ]:
#print the column names
print(df.columns)

In [ ]:
#look at the shape = (number of rows, number of columns
df.shape

In [ ]:
#index keys each row. can be anything that's unique for each row including tuples
df.index

In [ ]:
#access a particular row - iloc = integer location where integer = [0, number of rows - 1]
df.iloc[10]

In [ ]:
#access a particular row - loc = location by key in index
df.loc[3]

In [ ]:
#accessing multiple rows
df.loc[3:10]

In [ ]:
#access multiple rows and multiple columns
df.loc[3:10, ('Weekday', 'ScanCount')]

In [ ]:
#can also change values
print(df.loc[3, 'Weekday'])
#--> should give "Friday"

df.loc[3, 'Weekday'] = 'Sunday'
print(df.loc[3, 'Weekday'])
#--> should give "Sunday"

df.loc[3, 'Weekday'] = 'Friday'
print(df.loc[3, 'Weekday'])
#undo back to "Friday"

### WARNING: Generally avoid chained assignments

df.loc[3]['Weekday'] = 'Monday'

-> Unpredictable results depending on whether df.loc[3] returns a view of the original dataframe df, or a copy. In case of a copy, you'll modify the copy and not the original dataframe. Best to use:

df.loc[3, 'Weekday'] = 'Monday'

In [ ]:
#convert pandas dataframe to numpy array
np.array(df)

In [ ]:
#access a particular column - returns a pandas Series
df['TripType']

In [ ]:
#access multiple columns - returns a pandas dataframe
cols = ['TripType', 'VisitNumber']

df[cols].head()

In [ ]:
#look at first few rows of the two columns
df[['TripType', 'VisitNumber']].head()

In [ ]:
#find unique values in a column
df['TripType'].unique()

In [ ]:
#find unique values and counts for values in a column
df['TripType'].value_counts()

In [ ]:
#unique values and counts for different column
df['Upc'].value_counts()

In [ ]:
#histogram for values in column
df['Upc'].hist()

In [ ]:
#alternate way of making the histogram
plt.hist(df['Upc'].tolist());
plt.yscale('log')

In [ ]:
#plot number of unique values for "Upc" in descending order
plt.figure(figsize=(8,8))
plt.plot(df['Upc'].value_counts().tolist(), 'p')

In [ ]:
df['Upc'].value_counts().head(100)

In [ ]:
#often useful to look at pivot tables

pd.pivot_table(df, index = 'DepartmentDescription', columns = 'Weekday', values = 'TripType', aggfunc=lambda x: len(x))

The table above shows the number of trips made on each day to each department. In other words, each entry in the table is the number of rows in train for that day and that department.

Some notes:

* Why would this be useful - we are trying to classify trips. While we don't know Walmart's detailed rules for assigning TripTypes, it is reasonable to assume that the departments items are bought from have a lot of bearing on their rules. Tables like these help us understand correlations between the day of week and what is being bought.

* Often such a table should be "normalized". One example is dividing each row by the total sum to get a %. This would help us compare two different departments. Otherwise the absolute numbers have very different scales across different rows.

* Another way of normalizing is across columns i.e. divide each element by the sum of the values along its column. This would give us a % distribution of departments bought from on a given day. This is also tricky since the same trip might involve more than one department and we are counting each item bought once. In other words, we are ignoring the "ScanCount" column.

We'll do more exploratory analysis below but the idea here it to show an example of a pivot table.

In [ ]:
#group bys

trip_dist_by_day = df.groupby(['Weekday', 'TripType']).agg({'VisitNumber': lambda x: len(np.unique(x))})
print(trip_dist_by_day.head(10))

### Note: There is a lot of other pandas functionality that hasn't been shown above but this serves as a good introduction to what one can do with pandas.

**Additional reference**: https://pandas.pydata.org/pandas-docs/version/0.25/

# Problem Understanding

This is the part where we begin to do some exploratory analysis to understand the data we are working with. There are two general ways in which people approach a dataset:

1. Explore the data and build your intuition. This involves asking a sequence of questions and answering them with the data. This step also informs the features that should be computed to build a model.

2. The second approach is to use machine learning techniques to understand the data. This involves building a model as quickly as possible and probing the model to understand what's going on in the data.

### Approach 1: Exploratory Analysis -> Model Building

In [ ]:
df.head()

One advantage with this data is that we are all familiar with grocery shopping. Often a data scientist encounters a dataset which is very alien - for example, we could have used a particle physics dataset here and that would involve learning at least a few basic things about the underlying physics.

Here, we have data on trips taken by individuals to a Walmart store. Each trip has a unique id in **VisitNumber**.

**Question 1**: How many total visits do we have?

In [ ]:
len(df['VisitNumber'].unique())

Each row represents one product bought in each trip. So there are as many rows as there are products bought (there can be several quantities of each product bought but we won't worry about that now).

**Question 2**: How many total products were bought? 

In [ ]:
df.shape[0]

**Question 3**: How many unique products were bought per visit?

In [ ]:
df.shape[0] / len(df['VisitNumber'].unique())

Now let's use our intuition. What can we record when we go shopping?

**Which day we shopped on (1 column)**: Weekday

**What items we bought (3 columns)**: we have 3 columns describing this. At the coarsest level, the UPC code (bar code) tells us exactly what item was bought. Finelineno is a more coarse description of the type of product bought. Maybe it's something like fruit juice vs biscuits (although we don't know what each value denotes). At the coarsest level, we have DepartmentDescription which is pretty self-explanatory. 

**How many items of a particular type were bought (1 column)**: Did we buy one bottle of orange juice or 3? This is captured by ScanCount. In some cases, items might be returned, in which we have ScanCount == -1.

Capturing this data in a flat structure is a bit tricky so instead of having a list of items and their counts on each row, we have one row for each distinct product bought in each trip. To keep track of which trip a product belongs to, we have the **VisitNumber** column as stated above. This is a unique id for each shopper's trip.

Lastly, we have the **TripType** column which is an internal flag set by Walmart to classify each trip. Our task is to predict this tag for each visit. At this stage, we should ask a few careful questions. In the train set and test set, each row has **one** product. Is our task to predict the TripType for each *product*? That doesn't really make sense. I might be on a food shopping trip but buy a couple of batteries on the side. Clearly, the row with the same VisitNumber but the battery product should be still classified as a food shopping trip. This seems to imply that we should aggregate all the products for a trip, flatten it into some reasonable data structure and use that to predict the TripType.

**Question 4**: Do all the entries/rows for the same visit number have the same trip type? Or do we have cases where visit number N (for some N) has different trip types based on the product?

In [ ]:
df.groupby('VisitNumber').agg({'TripType': lambda x: len(np.unique(x))})['TripType'].unique()

In [ ]:
print(df[['VisitNumber', 'TripType']].drop_duplicates()['VisitNumber'].shape)
print(df[['VisitNumber', 'TripType']].drop_duplicates()['VisitNumber'].drop_duplicates().shape)

**Question 5**: How many unique trip types are there? What are their counts?

In [ ]:
df['TripType'].value_counts()

**Question 6**: For each trip, compute the total number of unique products (UPC), the total count of products ("ScanCount"), total number of unique Fileno and total number of unique DepartmentDescriptions

In [ ]:
df.head()

We have an issue with DepartmentDescription. Are some values floats?

**Question 7**: Debug this issue

In [ ]:
df['DepartmentDescription'].dtype

#A dtype of 'O' indicates 'object' i.e. that we might have fixed types.

In [ ]:
df['DepartmentDescription'].unique()

In [ ]:
#there are some nan values. let's confirm this
df['DepartmentDescription'].isnull().sum()

In [ ]:
#let's replace these with a string "null"
df['DepartmentDescription'].fillna('nan', inplace=True)

In [ ]:
unique_counts = df.groupby(['VisitNumber']).agg({'Upc': lambda x: len(np.unique(x)),
                                                 'FinelineNumber': lambda x: len(np.unique(x)),
                                                 'DepartmentDescription': lambda x: len(np.unique(x)),
                                                 'ScanCount': np.sum
                                                })

In [ ]:
unique_counts.head()

We have some statistics for each visit. But we don't have the TripType anymore. 

**Question 8**: Use the original dataframe, df and add a column to unique_counts that contains the TripType.

Hint: Use pd.concat or pd.merge (also see pd.DataFrame.set_index)

In [ ]:
df[['VisitNumber', 'TripType']].drop_duplicates().set_index('VisitNumber').head()

In [ ]:
df_collapsed = pd.concat([unique_counts, df[['VisitNumber', 'TripType']].drop_duplicates().set_index('VisitNumber')], axis=1)
df_collapsed

This is an example of a dataset that could be used to train a model. To be clear, the features are only counts. We have lost a lot of information - the (probability) distribution of UPC, FilelineNumber, DepartmentDescription and ScanCount have been collapsed into just overall counts. We can use this table to create a reasonable baseline model. Let's do that - we haven't gotten into the details of random forests yet but we can still use it as an example.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=100,                           
                               n_jobs=10)
model.fit(df_collapsed.drop('TripType', axis=1), df_collapsed['TripType'])

Note: df.drop(COLUMN_NAME, axis=1) removes the column from the dataframe. So we are using everything except TripType as the first argument to model.fit which specifies the input features and only TripType as the second argument which specifies the target variable.

**NOTE**: If this seems opaque and you don't understand it, please let us know in the class.

Now, a very important aspect of a problem is the evaluation metric. This is the quantity that is used to measure the performance of your model. In real-world problems, often the metric is chosen to reflect real-life priorities. Often, there are multiple metrics of interest (precision, recall, accuracy etc.). Here we can go to the Kaggle page and see that the metric of interest is the **multi-class logarithmic loss**:

$$-\frac{1}{N} \Sigma_{i=1}^N \Sigma_{j=1}^M y_{ij} \log(p_{ij})$$

where:

$N$ = number of rows

$M$ = number of trip types

$
y_{ij} =
\begin{cases}
1,  \text{ if ith row has trip type j}\\
0,   \text{ otherwise} \\
\end{cases}$

$p_{ij} = \text{probability of the ith row being of trip-type j}$

Note that $y_{ij}$ are labels and $p_{ij}$ are your model's predictions. Also, note that for a fixed row $i$, only one of $y_{ij} = 1$ and the rest are 0. So the contribution of that row to the cost is $-\log(p_{ij})$. In other words, if the model very confidently predicts the correct class, $p_{ij} \approx 1 \implies -\log(p_{ij}) \approx 0$ i.e. there's no contribution to the cost. On the other hand, if the model predicts the correct class' probability with very low confidence i.e. $p_{ij}\approx 0 \implies -log(p_{ij}) \approx \infty$ so we get a very high loss.

So even one bad prediction can really throw off your loss function and give a large value.

https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/overview/evaluation

In [ ]:
#a random forest also predicts the probabilities of belonging to each class

model.predict_proba(df_collapsed.head(10).drop('TripType', axis=1)).shape

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
log_loss(df_collapsed['TripType'], model.predict_proba(df_collapsed.drop('TripType', axis=1)),
         eps=10**(-15),
         normalize=True)

In [ ]:
model.predict(df_collapsed.head(10).drop('TripType', axis=1))

In [ ]:
model.classes_

If you look at the public leaderboard here:

https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/leaderboard

this model should get you to ~610 out of ~1050 participants. But not so fast! Recall our discussion of splitting the data into train and validation. We didn't do that here!! We just trained on a dataset and looked at our loss on it. The model almost surely is over-fitting and giving us a better (lower) loss that would not generaize to a new dataset. So we should split the train set into a true train set that we'll use for model building and a validation set to ensure we are not overfitting.

The question then is: how should we split the train set into train and validation? Should we just randomly sample, say 70% of the rows and call it train and 30% and call it test? 

**Question 9**: Can you think of a problem with this splitting strategy?

The problem is that we might end up splitting some VisitNumbers (trips) where a part of the trip goes to train and another part goes to the validation set. Clearly, we don't want that. So instead we should look at visit numbers and pick, say 70% and put them in train and the other 30% into test. Let's do that and persist the datasets.

**A subtle point**:

Generally you want to split the data into 3 pieces:

Train: Used to train your model

Validation: Evaluate your trained model on the validation set to measure your loss/metrics. Then go back to the train set and tweak the features and model to make this better. Compare the loss/metrics on train and validation to ensure your model is not overfitting.

Test: After you are confident your model building loop between train and validation are complete, evaluate your model here to get an estimate of how it would perform on new data.

One should never even look at the test data during the training process. There's a possibility of **leakage** i.e. information passing from the test data to your model-building process. Sometimes leakage is explicit, where you accidentally measure a quantity in the test dataset and use it in your model. Sometimes it is implicit - the sheer act of measuring the model's performance on the test set, making changes to your model and only accepting changes that improve the metric lead you down a biased path during model building.

Splitting your dataset into train, validation and test can be tricky. It doesn't matter what machine learning *technique* one uses for model building, without the right strategy, your model will not generalize to new data i.e. it won't perform as well on new data as it did on the train data.

In [ ]:
unique_visit_numbers = df['VisitNumber'].unique()
unique_visit_numbers.sort() #see explanation below - this will eventually turn out to be troublesome

In [ ]:
print(len(unique_visit_numbers))

In [ ]:
N_visits = len(unique_visit_numbers)

TRAIN_PERC = 0.80
TRAIN_CUTOFF = int(TRAIN_PERC * N_visits)

VAL_PERC = 0.10
VAL_CUTOFF = int((TRAIN_PERC + VAL_PERC) * N_visits)

TEST_PERC = 0.10

train_visit_numbers = unique_visit_numbers[:TRAIN_CUTOFF]
val_visit_numbers = unique_visit_numbers[TRAIN_CUTOFF:VAL_CUTOFF]
test_visit_numbers = unique_visit_numbers[VAL_CUTOFF:]

In [ ]:
#number of visits in each set
print(len(train_visit_numbers))
print(len(val_visit_numbers))
print(len(test_visit_numbers))

In [ ]:
#verify no overlap between visit numbers in 3 sets
print(len(set(train_visit_numbers).intersection(val_visit_numbers)))
print(len(set(train_visit_numbers).intersection(test_visit_numbers)))
print(len(set(val_visit_numbers).intersection(test_visit_numbers)))


* We have chosen to put 80% of the visit numbers in train, 10% in validation and 10% in test

* Instead of randomly choosing 80% of the visit numbers to be train, we ordered/sorted the visit numbers and chose the first 80% to be train, the next 10% to be val and the last 10% to be test.

* At this stage, I am not sure the above strategy is appropriate or whether I should choose the visit numbers randomly. My motivation for choosing this strategy was as follows: If time is involved in your dataset - either as an explicit column or implicitly, then one should choose the 3 datasets to be contiguous in time. Randomly choosing rows would give your bad generalization estimates because one would be using "future" data to predict "past" data i.e. one would have more information during model building than during actually running the model in production. It is possible (but by no means, certain) that VisitNumber values are ordered in time - probably by check-out time. If that is the case, then this strategy, while making it harder to build a predictive model, will make it very likely that if our model performs well on the test set, it'll perform well on real data.

[Optional Reading]: [TBC]

* A kaggle note: The motivation on kaggle is to win i.e. get the best possible loss/metric on Kaggle's test set. As a competitor, you want to ensure that the loss/metric you measure on your hold-out validation or test set, is close to the score that Kaggle's leaderboard shows you when you make a submission. This is an indirect measure of how close your validation set is to to Kaggle's validation set (close in terms of distributions etc.). A couple of examples will suffice to make the point:

* Example 1: Good generalization: In the Walmart problem we are solving, check out the public and private leaderboards here: https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/leaderboard. The ranking of the top 9 competitors is exactly the same on the public and private leaderboard. Moreover, the loss values don't differ by much between the two leaderboards.

* Example 2: Bad generalization: https://www.kaggle.com/c/afsis-soil-properties/leaderboard. If you look at the private leaderboard, you'll see arrows under $\Delta\text{pub}$ that tell you how much someone's position changed compared to the public leaderboard. Positions have moved by hundreds of positions! If you were at the top on the public leaderboard, you were 516th(!) on the private leaderboard. This can happen mainly due to two reasons (1) the competitor was overfitting on the validation set used to compute the public leaderboard and not doing cross-validation properly. If this is the case, it seems many people fell into this trap instead of just trusting the scores they got on their private validation and test sets. (2) Kaggle had created a test set that had markedly different distributions or properties compared to the validation set.

In [ ]:
#persist these values so we don't have re-compute this each time
import pickle

In [ ]:
pickle.dump({'train': train_visit_numbers,
             'val': val_visit_numbers,
             'test': test_visit_numbers}, open(f'{DATA_PATH}/cv_visit_numbers.pkl', 'wb'))

In [ ]:
visit_numbers = pickle.load(open(f'{DATA_PATH}/cv_visit_numbers.pkl', 'rb'))

train_visit_numbers = visit_numbers['train']
val_visit_numbers = visit_numbers['val']
test_visit_numbers = visit_numbers['test']

In [ ]:
#create dataframes for each set
df = pd.read_csv(TRAIN_PATH) #read file again in case we did some operations that could cause leakage (fill nulls etc.)

In [ ]:
df_train = df[df['VisitNumber'].isin(train_visit_numbers)]
df_val = df[df['VisitNumber'].isin(val_visit_numbers)]
df_test = df[df['VisitNumber'].isin(test_visit_numbers)]

In [ ]:
df_train.to_feather(f'{DATA_PATH}/df_raw_train.feather')
df_val.reset_index(drop=True).to_feather(f'{DATA_PATH}/df_raw_val.feather')
df_test.reset_index(drop=True).to_feather(f'{DATA_PATH}/df_raw_test.feather')

# END

Tomorrow, we'll use these datasets to create features that can be used in **linear** models. The main workhorse here is **logistic regression** and while it is a very simple technique, it can be extremely powerful.